In [1]:
import torch
import torchaudio
from cnn import CNNNetwork
from dataset import BirdDataset
from torch.utils.data import DataLoader
from train import SAMPLE_RATE, NUM_SAMPLES

class_mapping = [
    "kaka",
    "kiwi",
    "kokako",
    "ruru",
    "tui",
    "tieke"
]

In [2]:
def predict(model, input, target, class_mapping):
    model.eval() # switches the model to prediction, use model.train() to switch to training mode
    
    with torch.no_grad():
        predictions = model(input) # -> [[0.1, 0.0, ..., 0.7]]
        predicted_index = predictions[0].argmax(0)
        predicted = class_mapping[predicted_index]

        expected = class_mapping[target]
    
    return predicted, expected

In [3]:
# load back the model
cnn = CNNNetwork()
state_dict = torch.load("cnn.pth")
cnn.load_state_dict(state_dict)


C:\Users\Mitchell Pincham\AppData\Local\Temp\ipykernel_6860\2860347440.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("cnn.pth")


<All keys matched successfully>

In [4]:
# load urban sound validation dataset
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE, n_fft=1024,hop_length=512, n_mels=64)

dataset = BirdDataset(mel_spectrogram, "cpu", SAMPLE_RATE, NUM_SAMPLES)

In [10]:
# get a sample from the validaton dataset for inference

correct = 0
total = 0
for input, target in dataset:
    total += 1
    input.unsqueeze_(0)

    # make an inference
    predicted, expected = predict(cnn, input, target, class_mapping)

    if predicted == expected:
        correct += 1

print(correct)
print(total)

577
1193


In [ ]:
# make an inference
predicted, expected = predict(cnn, input, target, class_mapping)

print(f"Predicted = {predicted}, Expected = {expected}")

Predicted = tui, Expected = tui
